In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datasetsforecast.long_horizon import LongHorizon

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, RMoK, TSMixer

from utilsforecast.losses import mae, mse
from utilsforecast.evaluation import evaluate

D:\Anaconda\envs\neuralforecast-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_data(name):
    if name == "ettm1":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == "ettm2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == 'etth1':
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'
    elif name == "etth2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'

    return Y_df, val_size, test_size, freq

In [4]:
Y_df, val_size, test_size, freq = load_data('ettm1')
horizon = 96

models = [
    # RMoK
    RMoK(
        h=horizon,
        input_size=horizon,
        n_series=7,
        taylor_order=3,
        jacobi_degree=6,
        wavelet_function='mexican_hat',
        dropout=0.1,
        revine_affine=True,
        learning_rate=0.001,
        scaler_type='identity',
        max_steps=1000,
        early_stop_patience_steps=3),
    )
    TSMixer(h=horizon, input_size=3*horizon, n_series=7, max_steps=1000, early_stop_patience_steps=3),
    NHITS(h=horizon, input_size=3*horizon, max_steps=1000, early_stop_patience_steps=3),
]

nf = NeuralForecast(models=models, freq=freq)
nf_preds = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=None)
nf_preds = nf_preds.reset_index()

ettm1_evaluation = evaluate(df=nf_preds, metrics=[mae, mse], models=['RMoK', 'TSMixer', 'NHITS'])

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type          | Params | Mode 
--------------------------------------------------
0 | loss    | MAE           | 0      | train
1 | padder  | ConstantPad1d | 0      | train
2 | scaler  | TemporalNorm  | 0      | train
3 | dropout | Dropout       | 0      | train
4 | experts | ModuleList    | 138 K  | train
5 | gate    | Linear        | 388    | train
6 | softmax | Softmax       | 0      | train
7 | rev     | RevIN         | 14     | train
--------------------------------------------------
139 K     Trainable params
0         Non-trainab

Epoch 999: 100%|██████████| 1/1 [00:07<00:00,  0.14it/s, v_num=148, train_loss_step=0.340, train_loss_epoch=0.340, valid_loss=0.402]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 999: 100%|██████████| 1/1 [00:07<00:00,  0.14it/s, v_num=148, train_loss_step=0.340, train_loss_epoch=0.340, valid_loss=0.402]

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.


...


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/marcopeix/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/torch/nn/functional.py:4024: UserWarning: The operator 'aten::upsample_linear1d.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403226260/work/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._C._nn.upsample_linear1d(input, output_size, align_corners, scale_factors)


Epoch 499: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s, v_num=152, train_loss_step=0.273, train_loss_epoch=0.273, valid_loss=0.419]

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


In [5]:
ettm1_evaluation = ettm1_evaluation.drop(['unique_id'], axis=1)\
                                   .groupby('metric')\
                                   .mean()\
                                   .reset_index()

ettm1_evaluation

,metric,RMoK,TSMixer,NHITS
0,mae,0.355421,0.333564,0.356006
1,mse,0.332293,0.290767,0.342725
